In [ ]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import os
import warnings
warnings.filterwarnings('ignore')
print(os.listdir('../input/elo-stack-1'))

In [ ]:
path='../input/elo-merchant-category-recommendation/'
df_train=pd.read_csv(os.path.join(path,'train.csv'))
df_test=pd.read_csv(os.path.join(path,'test.csv'))
path='../input/elo-classification-2'
prob_train=pd.read_csv(os.path.join(path,'prediction_PROB_LGB_half_0.9.csv'))
prob_test=pd.read_csv(os.path.join(path,'submission_PROB_LGB_half_0.9.csv'))
# path='../input/elo-regression-1'
# rmse_train=pd.read_csv(os.path.join(path,'prediction_LGB_full_3.64544.csv'))
# rmse_test=pd.read_csv(os.path.join(path,'submission_LGB_full_3.64544.csv'))
path='../input/elo-stack-1'
rmse_train=pd.read_csv(os.path.join(path,'prediction.csv'))
rmse_test=pd.read_csv(os.path.join(path,'submission.csv'))
path='../input/elo-regression-4'
cut_rmse_train=pd.read_csv(os.path.join(path,'prediction_LGB_full_3.8.csv'))
cut_rmse_test=pd.read_csv(os.path.join(path,'submission_LGB_full_3.8.csv'))
prediction_train=pd.DataFrame({'target':df_train['target'].values,'prob':prob_train['target'].values,'rmse':rmse_train['target'].values,'rmse_cut':cut_rmse_train['target'].values})
prediction_test=pd.DataFrame({'prob':prob_test['target'].values,'rmse':rmse_test['target'].values,'rmse_cut':cut_rmse_test['target'].values})

In [ ]:
df_train['outliers']=0
df_train.loc[df_train['target']<-30,'outliers']=1
rskf=StratifiedKFold(5,shuffle=True)
val_pr=np.zeros(len(df_train))
itog=[]
for train_index,val_index in rskf.split(df_train,df_train['outliers'].values):
    thresholds = []
    for thresh in np.arange(0.01, 1, 0.01):
        thresh = np.round(thresh, 2)
        pr=np.where(prediction_train['prob'].loc[val_index]>thresh,prediction_train['rmse'].loc[val_index],prediction_train['rmse_cut'].loc[val_index])
        res = np.sqrt(mean_squared_error(pr,prediction_train['target'].loc[val_index].values))
        thresholds.append([thresh, res])
    thresholds.sort(key=lambda x: x[1], reverse=False)
    best_thresh = thresholds[0][0]
    itog.append(best_thresh)
np.sqrt(mean_squared_error(np.where(prediction_train['prob']>np.mean(itog),prediction_train['rmse'],prediction_train['rmse_cut']),prediction_train['target'].values))

In [ ]:
test_pr=np.where(prediction_test['prob']>np.mean(itog),prediction_test['rmse'],prediction_test['rmse_cut'])
sub_df=pd.DataFrame({'card_id':df_test['card_id'].values})
sub_df['target']=test_pr
sub_df.to_csv("submission.csv", index=False)